# Linear Support Vector Machines (SVMs)
线性SVM是一个标准的大型分类任务，它是一种$ \begin{equation}
    f(v) := \lambda\, R(w) +
    \frac1n \sum_{i=1}^n L(v;x_i,y_i)
    \label{eq:regPrimal}
    \ .
\end{equation} $ 的线性模型，损失函数如下：
$$L(w;x,y) := \max \{0, 1-y w^T x \}.$$ 
默认情况下SVM使用的是L2正则化，我们也可以设置成L1.  
输入数据给分类器会输出一个类别标签，这相当于一个类似于Sigmoid的函数作用。模型的预测结果是基于 $ w^T x $的值，如果$ w^T x $ 如果大于0 输出＋1 否则输出－1.
## Examples
下面的代码演示如何加载数据集，我们通过调用SVMWithSGD对象的静态方法来训练测试数据集，并且计算预测误差。

In [1]:
val PATH = "file:///Users/lzz/work/SparkML/"
import org.apache.spark.mllib.classification.{SVMModel, SVMWithSGD}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.util.MLUtils

// Load training data in LIBSVM format.
val data = MLUtils.loadLibSVMFile(sc, PATH+"data/mllib/sample_libsvm_data.txt")

// Split data into training (60%) and test (40%).
val splits = data.randomSplit(Array(0.6, 0.4), seed = 11L)
val training = splits(0).cache()
val test = splits(1)

// Run training algorithm to build the model
val numIterations = 100
val model = SVMWithSGD.train(training, numIterations)

// Clear the default threshold.
model.clearThreshold()

// Compute raw scores on the test set.
val scoreAndLabels = test.map { point =>
  val score = model.predict(point.features)
  (score, point.label)
}

// Get evaluation metrics.
val metrics = new BinaryClassificationMetrics(scoreAndLabels)
val auROC = metrics.areaUnderROC()

println("Area under ROC = " + auROC)

// Save and load model
model.save(sc, "myModelPath")
val sameModel = SVMModel.load(sc, "myModelPath")

Area under ROC = 1.0


SVMWithSGD.train() 该方法默认采用的是L2正则化。如果我们想配置这些算法，我们可以自定义 SVMWithSGD 进一步通过创建新对象直接调用setter方法。其它 MLlib 也是支持自定义配置方法也是一样的。例如，下面的代码设置 SVM 的正则化参数 0.1 并迭代 200 次运行。

In [3]:
import org.apache.spark.mllib.optimization.L1Updater

val svmAlg = new SVMWithSGD()
svmAlg.optimizer.
  setNumIterations(200).
  setRegParam(0.1).
  setUpdater(new L1Updater)
val modelL1 = svmAlg.run(training)